In [ ]:
import torch
import torch.nn as nn
import numpy as np
import math
from scipy.interpolate import interp1d
import csv
from datetime import datetime, timedelta
from plyfile import PlyData, PlyElement
import pickle
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.dates as mdates
from matplotlib import cm
from matplotlib.colors import Normalize
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import glob
import pandas as pd
from scipy.spatial import ConvexHull
from scipy.optimize import brentq
import random

from ThermoONet_architecture import Branch
from ThermoONet_architecture import SELayer
from ThermoONet_architecture import SELayer_w
from ThermoONet_architecture import Branch1
from ThermoONet_architecture import Branch2
from ThermoONet_architecture import Branch3

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
num_sensors = 128
branch = Branch(num_sensors).cuda()
# Read the trained network
branch = torch.load("Network/network_accurate.pkl")

# Z-score parameters
mean, std = 0.5216620832565283, 0.16037000058696135
mean_p1, std_p1 = 5.587696141721031, 1.8642376418421138
mean_p2, std_p2 = 0.9012508495757381, 0.057731154082425275
mean_p3, std_p3 = 0.011181149021523971, 0.004995541538499874
mean_p4, std_p4 = 16.542181636792183, 12.499430941576392
mean_p5, std_p5 = 16.524861547108635, 12.931041392620573
mean_p6, std_p6 = 0.45722424339279366, 0.306981614080625
mean_p7, std_p7 = 0.058400414325008906, 0.030512679311208945
mean_p8, std_p8 = 0.06281489233220064, 0.034735621144138

In [1]:
"""Cometary size estimation from SOHO/SWAN observations"""
# files1: Water production rate from SOHO/SWAN
# files2: Orbit from JPL corresponding to SOHO/SWAN

class Size_inverse():
    def __init__(self, file1, file2):
        self.file1, self.file2 = file1, file2
    
    def Z_Score(self, x):
        mean = np.mean(x)
        std = np.std(x)
        return mean, std, (x-mean)/std

    def Minmax(self, x):
        min_x = np.min(x)
        max_x = np.max(x)
        x_norm = x/max_x
        return min_x, max_x, x_norm

    # Some functions
    def ZHK(self, p_global3, p_ice3, u):
        mb = 3e-26
        kb = 1.38e-23
        psi = 1/(1+0.14*p_global3/0.001)

        a = 3.23e+12
        b = 6134.6
        c0 = 0.146
        c1 = 0.854
        c2 = 57.78
        c3 = 11580

        Pv = a*np.exp(-b/(u))
        alpha = c0+c1/(1+np.exp(c2-c3/(u)))
        Z_HK = psi*p_ice3*alpha*Pv*np.sqrt(mb/(2*np.pi*kb*(u)))
        return Z_HK

    def equator_rotation(self, theta, omega, pos):
        cos_omega = np.cos(omega)
        sin_omega = np.sin(omega)
        rotation_matrices = np.array([
                            [cos_omega, sin_omega, np.zeros_like(cos_omega)],
                            [-sin_omega, cos_omega, np.zeros_like(cos_omega)],
                            [np.zeros_like(cos_omega), np.zeros_like(cos_omega), np.ones_like(cos_omega)]])

        rotation_matrix_z = rotation_matrices.transpose(2, 0, 1)

        cos_theta = np.cos(np.pi/2-theta)
        sin_theta = np.sin(np.pi/2-theta)
        rotation_matrices = np.array([
                            [cos_theta, np.zeros_like(theta), -sin_theta],
                            [np.zeros_like(theta), np.ones_like(theta), np.zeros_like(theta)],
                            [sin_theta, np.zeros_like(theta), cos_theta]])

        rotation_matrix_y = rotation_matrices.transpose(2, 0, 1)

        pos_new = np.sum(rotation_matrix_y*np.sum(rotation_matrix_z*pos.reshape(len(pos), 1, 3), \
                                                  axis=-1).reshape(len(pos), 1, 3), axis=-1)
        return pos_new

    def generate_ellipsoid_triangulation(self, a, b, c, num_points=16):
        u = np.linspace(0, 2*np.pi, num_points)
        v = np.linspace(0, np.pi, num_points)

        x = a*np.outer(np.cos(u), np.sin(v))
        y = b*np.outer(np.sin(u), np.sin(v))
        z = c*np.outer(np.ones_like(u), np.cos(v))

        points = np.array([x.flatten(), y.flatten(), z.flatten()]).T

        hull = ConvexHull(points)
        triangles = hull.simplices

        return points, triangles

    def utc_to_julian(self, utc_time):
        if not isinstance(utc_time, datetime):
            raise ValueError("must datetime object")
        julian_day = utc_time.toordinal()+1721424.5+(utc_time.hour + utc_time.minute/60+utc_time.second/3600)/24
        return julian_day

    def julian_to_utc(self, julian_day):
        if not isinstance(julian_day, (float, int)):
            raise ValueError("must float or int")

        z = int(julian_day+0.5)
        f = julian_day+0.5-z
        if z < 2299161:
            a = z
        else:
            alpha = int((z-1867216.25)/36524.25)
            a = z+1+alpha-int(alpha/4)

        b = a+1524
        c = int((b-122.1)/365.25)
        d = int(365.25*c)
        e = int((b-d)/30.6001)

        day = b-d-int(30.6001 * e)+f
        month = e-(1 if e < 14 else 13)
        year = c-(4716 if month > 2 else 4715)

        hour = int((f*24))
        minute = int(((f*24)-hour)*60)
        second = int((((f*24)-hour)*60-minute)*60)

        return datetime(year, month, int(day), hour, minute, second)

    def cartesian_to_spherical(self, x, y, z):
        r = math.sqrt(x**2+y**2+z**2)
        theta = math.acos(z/r)
        phi = math.atan2(y, x)
        if phi<0:
            phi = 2*np.pi+phi
        return r, theta, phi

    def normal_vector(self, A, B, C):
        AB = B-A
        AC = C-A
        nv = np.cross(AB, AC)
        return nv/np.sqrt(np.sum(nv**2))

    def center_vector(self, A, B, C):
        Cx = (A[0]+B[0]+C[0])/3
        Cy = (A[1]+B[1]+C[1])/3
        Cz = (A[2]+B[2]+C[2])/3
        return np.array([Cx, Cy, Cz])

    def area_surface(self, A, B, C):
        v1 = B-A
        v2 = C-A
        cross_product = np.cross(v1, v2)
        area = 0.5*np.linalg.norm(cross_product)
        return area

    def fl(self, beta, yita, nvs):
        # calculate the radiation flux function of every facet in spin axis coordinate system

        # beta: the angle between the Sun and z-axis
        # yita: the angle between the Sun and x-axis
        # nvs: the normal vectors of facets

        N = 128
        sun = np.array([np.sin(beta)*np.cos(yita), np.sin(beta)*np.sin(yita), np.cos(beta)])
        if np.sqrt(sun[0]**2+sun[1]**2)==0:
            lo_sun = np.arccos(0)
        else:
            lo_sun = np.arccos(sun[0]/np.sqrt(sun[0]**2+sun[1]**2))
        if yita > 180*np.pi/180:
            if np.sqrt(sun[0]**2+sun[1]**2)==0:
                lo_sun = 2*np.pi-np.arccos(0)
            else:
                lo_sun = 2*np.pi-np.arccos(sun[0]/np.sqrt(sun[0]**2+sun[1]**2))
        sun_xy = np.sqrt(sun[0]**2+sun[1]**2)
        sun = np.array([sun_xy*np.cos(lo_sun-np.linspace(0, 2*np.pi, N)), \
                        sun_xy*np.sin(lo_sun-np.linspace(0, 2*np.pi, N)), \
                        sun[2]*np.ones(N)]).T

        function = np.dot(nvs, sun.T)
        function[function<0] = 0

        flist_ellipsoid = interp1d(np.linspace(0, 2*np.pi, N), function, kind='linear', axis=1)

        return flist_ellipsoid
    
    def water_production(self, ars, nvs, df_list, orbit_whole, params):
        orbit_co_whole = []
        for j in range(len(orbit_whole)):
            orbit_j = orbit_whole[j]
            theta = params[10]
            omega_1 = params[9]
            orbit_co_mid = -self.equator_rotation(np.ones(len(orbit_j))*theta*np.pi/180, \
                                                 np.ones(len(orbit_j))*omega_1*np.pi/180, orbit_j[:, 1:4])

            orbit_co = []
            for i in range(len(orbit_j)):
                orbit_co.append(self.cartesian_to_spherical(orbit_co_mid[i, 0], orbit_co_mid[i, 1], orbit_co_mid[i, 2]))
            orbit_co = np.concatenate((df_list[j][:, 0][:, None], orbit_j[:, 0][:, None], np.array(orbit_co)), axis=-1)

            orbit_co_whole.append(orbit_co)
        orbit_co_whole = np.concatenate(orbit_co_whole, axis=0)
        orbit_co_whole = orbit_co_whole[orbit_co_whole[:, 0].argsort()]

        num_p = 12
        num = len(orbit_co_whole)

        zsj = 0
        yita_list = np.linspace(0, 2*np.pi, num_p)
        f_data = []
        for i in range(num):
            flist = self.fl(orbit_co_whole[i, -2], orbit_co_whole[i, -1], nvs)

            for j in range(num_p):
                f_for = flist(np.mod(np.linspace(0, 2*np.pi, 128)-yita_list[j], 2*np.pi))
                f_data.append(f_for.reshape(len(f_for), 128))
            zsj = zsj+1
        f_data = np.array(f_data).reshape(zsj, num_p, len(nvs), 128)
        f_test = torch.tensor(f_data).float().cuda()

        omega_spin = 2*np.pi/(10*3600)

        Te_list = []
        p_list = []
        deep_list = []
        sign_arr = np.sign(orbit_co_whole[:, 0])
        time_perihe = np.where(sign_arr[:-1] != sign_arr[1:])[0]+0
        for i in range(num):
            if i <= time_perihe:
                zsj = -np.exp(np.linspace(orbit_co_whole[time_perihe, 2]/1.496e+8*params[0], \
                                          np.max(orbit_co_whole[:, 2]/1.496e+8)*2, 100))
                zsj = (zsj-np.min(zsj))/(np.max(zsj)-np.min(zsj))*(params[1]-params[4])+params[4]
                zsj_interp = interp1d(np.linspace(min(orbit_co_whole[:, 0]), \
                                                  orbit_co_whole[time_perihe, 0], 100).flatten(), zsj.flatten(), kind='linear')
            else:
                zsj = -np.exp(-np.linspace(orbit_co_whole[time_perihe, 2]/1.496e+8*params[2], \
                                           np.max(orbit_co_whole[:, 2]/1.496e+8)*2, 100))
                zsj = (zsj-np.min(zsj))/(np.max(zsj)-np.min(zsj))*(params[3]-params[4])+params[4]
                zsj_interp = interp1d(np.linspace(orbit_co_whole[time_perihe, 0], \
                                                  max(orbit_co_whole[:, 0]), 100).flatten(), zsj.flatten(), kind='linear')

            p_global3 = zsj_interp(orbit_co_whole[i, 0])
            deep_list.append(p_global3)

        deep = np.array(deep_list)
    
        for i in range(num):
            Guy_list = []
            p_global1 = np.sqrt(np.sqrt((1-0.05)*1357/(orbit_co_whole[i, 2]/1.496e+8)**2))
            p_global = [p_global1, 1, deep[i]]
            p_dust = [params[8], params[6]]
            p_ice = [8.388111692959722, params[7], params[5]]

            p_ice[0] = np.sqrt((1-p_ice[-1])*p_dust[0]**2+p_ice[-1]*2100*920*omega_spin)

            p_data = np.array([p_global[0], p_global[1], p_global[2], p_dust[0], p_ice[0], p_ice[2], p_dust[1], p_ice[1]])
            Te_list.append(p_data[0]/(p_data[1]*5.67e-8)**(1/4))
            p_data = ((p_data-np.array([mean_p1, mean_p2, mean_p3, mean_p4, mean_p5, mean_p6, mean_p7, mean_p8])) \
                           /np.array([std_p1, std_p2, std_p3, std_p4, std_p5, std_p6, std_p7, std_p8]))
            p_list.append(p_data)
        p_data = np.array(p_list)
        Te = np.array(Te_list).reshape(num, 1)

        p_test = torch.tensor(p_data).float().cuda()
        output2 = branch.branch2(p_test[:, 1:8])
        output3 = branch.branch3(p_test[:, 0].unsqueeze(1))
        leakyrelu = nn.LeakyReLU(0.02)
        output23 = leakyrelu(branch.se(torch.mul(output2, output3)+0.001))

        WPR = []
        for i in range(num):
            Z_D_nump = []
            for j in range(num_p):
                output1 = branch.branch1(f_test[i, j])
                output = ((torch.mean(output1*output23[i], dim=-1)+0.01)*std+mean).detach().cpu().numpy()
                Guy = (output*Te[i]).reshape(1, len(f_for))
                Guy[np.where(Guy<0)] = -Guy[np.where(Guy<0)]

                Z_D = self.ZHK(deep[i], p_ice[-1], Guy)
                Z_D = np.sum(Z_D*ars.flatten(), axis=-1)
                Z_D_nump.append(Z_D)
            WPR.append(np.mean(np.array(Z_D_nump))*1e+6/3e-26)
        WPR_max = np.max(np.array(WPR))
        WPR = np.array(WPR/WPR_max)
        return WPR, WPR_max, deep

    # Loss function
    def objective_function(self, ars, nvs, df_list, orbit_whole, conditions, observed_data):
        solution, _, _ = self.water_production(ars, nvs, df_list, orbit_whole, conditions)
        mseloss = np.mean((np.abs(solution-observed_data)/observed_data))
        return mseloss

    # Simulated Annealing
    def simulated_annealing(self, ars, nvs, df_list, orbit_whole, observed_data, current_conditions, max_iter=20, \
                            initial_temp=100, final_temp=1, cooling_rate=0.8):
        loss_list = []
        current_list = []
        current_score = self.objective_function(ars, nvs, df_list, orbit_whole, current_conditions, observed_data)
        # print(current_score)
        best_conditions = current_conditions
        initial_conditions = current_conditions
        best_score = current_score

        temperature = initial_temp
        while temperature >= final_temp:
            best_Me_score = []
            for iteration in range(max_iter):
                while True:
                    new_conditions = [np.abs(current_conditions[i]+random.uniform(-initial_conditions[i]/3, initial_conditions[i]/3))
                                      for i in range(len(current_conditions))]
                    new_conditions[5] = np.abs(current_conditions[5]+random.uniform(-0.05, 0.05))
                    new_conditions[-2] = current_conditions[-2]+random.uniform(-45, 45)
                    new_conditions[-1] = current_conditions[-1]+random.uniform(-90, 90)

                    pan1 = 0.01<=new_conditions[0]<=2
                    pan2 = 0.001<=new_conditions[1]<=0.006
                    pan3 = 0.01<=new_conditions[2]<=2
                    pan4 = 0.001<=new_conditions[3]<=0.006
                    pan5 = 0.001<=new_conditions[4]<=0.006
                    pan6 = 0.01<=new_conditions[5]<=0.5
                    pan7 = 0.04<=new_conditions[6]<=0.07
                    pan8 = 0.04<=new_conditions[7]<=0.07
                    pan9 = 5<=new_conditions[8]<=12
                    pan10 = -90<=new_conditions[9]<=90
                    pan11 = 0<=new_conditions[10]<=360
                    pan12 = new_conditions[4]<new_conditions[1]
                    pan13 = new_conditions[4]<new_conditions[3]
                    pan14 = np.abs(new_conditions[1]-new_conditions[3])<=0.003
                    if pan1&pan2&pan3&pan4&pan5&pan6&pan7&pan8&pan9&pan10&pan11&pan12&pan13&pan14:
                        break

                new_score = self.objective_function(ars, nvs, df_list, orbit_whole, new_conditions, observed_data)
                # print('new_score:', new_score)
                loss_list.append(new_score)

                if new_score < current_score:
                    current_conditions = new_conditions
                    current_score = new_score
                else: 
                    if random.random() < np.exp((current_score-new_score)*1e+3/temperature):
                        current_conditions = new_conditions
                        current_score = new_score
                current_list.append(current_score)
                if current_score < best_score:
                    best_conditions = current_conditions
                    best_score = current_score
                else:
                    current_conditions = best_conditions
                    current_score = best_score

            # print('best:', best_score, best_conditions)

            temperature *= cooling_rate

        return best_conditions, best_score
    
    def cal(self):
        tab_files = []
        tab_files.append(self.file1)

        df_list = []
        for tab_file in tab_files:
            try:
                df = pd.read_csv(tab_file, sep='\t', header=None).to_numpy()
                df_mid = []
                for i in range(len(df)):
                    df_mid_1 = list(map(float, df[i, 0][21:].split()))
                    if df_mid_1[-1]<500 and df_mid_1[0]>-500:
                        df_mid.append(df_mid_1)
                df_list.append(np.array(df_mid)[::1])
            except Exception as e:
                print(f"{e}")
        if len(tab_files) != 1:
            df_whole = np.concatenate(df_list, axis=0)
        else:
            df_whole = df_list[0]
        df_whole = df_whole[df_whole[:, 0].argsort()]

        df_whole[:, -2] = df_whole[:, -2]*10**27
        df_whole = df_whole[::1, :]
        WPR_min, WPR_max, WPR_deal = self.Minmax(df_whole[:, -2])

        df_list_plot = []
        for tab_file in tab_files:
            try:
                df = pd.read_csv(tab_file, sep='\t', header=None).to_numpy()
                df_mid = []
                for i in range(len(df)):
                    df_mid_1 = list(map(float, df[i, 0][21:].split()))
                    if df_mid_1[-1]<500 and df_mid_1[0]>-500: # 1995_o1
                        df_mid.append(df_mid_1)
                df_list_plot.append(np.array(df_mid)[::1])
            except Exception as e:
                print(f"{e}")
        if len(tab_files) != 1:
            df_whole_plot = np.concatenate(df_list_plot, axis=0)
        else:
            df_whole_plot = df_list_plot[0]
        df_whole_plot = df_whole_plot[df_whole_plot[:, 0].argsort()]

        df_whole_plot[:, -2] = df_whole_plot[:, -2]*10**27
        df_whole_plot = df_whole_plot[::1, :]
        
        # Shape
        a1, b1, c1 = 1.5, 1, 1
        gui = np.max([a1, b1, c1])
        a, b, c = a1/gui, b1/gui, c1/gui
        vertices, triangles = self.generate_ellipsoid_triangulation(a, b, c)

        surface_element = []
        for i in range(len(triangles)):
            surface_element.append([vertices[int(triangles[i, 0]), :], vertices[int(triangles[i, 1]), :], \
                                    vertices[int(triangles[i, 2]), :]])
        surface_element = np.array(surface_element)

        surface_element_list = []
        for i in range(len(surface_element)):
            a2 = np.sum(surface_element[i, 0, :]-surface_element[i, 1, :])
            b2 = np.sum(surface_element[i, 0, :]-surface_element[i, 2, :])
            c2 = np.sum(surface_element[i, 1, :]-surface_element[i, 2, :])
            if a2!=0 and b2!=0 and c2!=0:
                surface_element_list.append(surface_element[i, :, :])
        surface_element = np.array(surface_element_list)

        # 计算法向量、面积
        nvs = []
        ces = []
        ars = []
        for i in range(len(surface_element)):
            nv = self.normal_vector(surface_element[i, 0, :], surface_element[i, 1, :], surface_element[i, 2, :])
            ce = self.center_vector(surface_element[i, 0, :], surface_element[i, 1, :], surface_element[i, 2, :])
            ar = self.area_surface(surface_element[i, 0, :], surface_element[i, 1, :], surface_element[i, 2, :])
            if np.dot(nv, surface_element[i, 0, :])<0:
                nv = -nv
            nvs.append(nv)
            ces.append(ce)
            ars.append(ar)
        nvs = np.array(nvs)
        ces = np.array(ces)
        ars = np.array(ars).reshape(len(surface_element), 1)
        
        tab_files = []
        tab_files.append(self.file2)

        orbit_whole = []
        looktabel = []
        for j in range(len(tab_files)):
            filename = tab_files[j]
            with open(filename, 'r') as file:
                lines = file.readlines()
                lines = [line.strip() for line in lines]
            orbit = []
            for i in range(len(lines)-1):
                orbit.append(list(map(float, lines[i+1][21:].split()))[1:])
            orbit = np.array(orbit)

            time_orb = []
            for i in range(len(lines)-1):
                year = list(map(int, lines[i+1][:4].split()))[0]
                month = list(map(int, lines[i+1][5:7].split()))[0]
                day = list(map(int, lines[i+1][8:10].split()))[0]
                hour = list(map(int, lines[i+1][11:13].split()))[0]
                minute = list(map(int, lines[i+1][14:16].split()))[0]
                second = list(map(int, lines[i+1][17:19].split()))[0]
                if second == 60:
                    second -= 1
                cos_time = datetime(year=year, month=month, day=day, hour=hour, minute=minute, second=second)
                time_orb.append(self.utc_to_julian(cos_time))
            time_orb = np.array(time_orb)[:, None]
            orbit = np.concatenate((time_orb, orbit), axis=-1)

            t_co = np.linspace(0, 1, len(orbit))
            orbit_interp = interp1d(t_co, orbit, axis=0)

            helio = np.sqrt(np.sum(orbit[:, 1:4]**2, axis=-1))/1.496e+8
            helio_interp = interp1d(t_co, helio, axis=0)

            num = len(df_list[j])
            t_data = []
            for i in range(num):
                def target(t):
                    return helio_interp(t)-df_list[j][i, 1]
                if df_list[j][i, 1] == np.min(df_list[j][:, 1]):
                    t_solution = t_co[np.argmin(helio)]
                else:
                    try:
                        if df_list[j][i, 0]<0:
                            t_solution = brentq(target, 0, t_co[np.argmin(helio)])
                        else:
                            t_solution = brentq(target, t_co[np.argmin(helio)], 1)
                    except:
                        aaaa = 1
                t_data.append(t_solution)
            t_data = np.array(t_data)

            orbit_use = orbit_interp(t_data)
            orbit_whole.append(orbit_use)

        theta = -50
        omega = 275
        orbit_co_whole = []
        for j in range(len(orbit_whole)):
            orbit_j = orbit_whole[j]
            orbit_co_mid = -self.equator_rotation(np.ones(len(orbit_j))*theta*np.pi/180, \
                                             np.ones(len(orbit_j))*omega*np.pi/180, orbit_j[:, 1:4])
            orbit_co = []
            for i in range(len(orbit_j)):
                orbit_co.append(self.cartesian_to_spherical(orbit_co_mid[i, 0], orbit_co_mid[i, 1], orbit_co_mid[i, 2]))
            orbit_co = np.concatenate((df_list[j][:, 0][:, None], orbit_j[:, 0][:, None], np.array(orbit_co)), axis=-1)
            orbit_co_whole.append(orbit_co)
        orbit_co_whole = np.concatenate(orbit_co_whole, axis=0)
        orbit_co_whole = orbit_co_whole[orbit_co_whole[:, 0].argsort()]
        
        size_list = []
        for i in range(30):
            observed_data = WPR_deal
            initial_conditions = np.array([1, 0.005, 1, 0.005, 0.003, 0.1, 0.05, 0.05, 7, 0, 180])
            estimated_conditions, score = self.simulated_annealing(ars, nvs, df_list, orbit_whole, observed_data, initial_conditions)

            water_c_test, max_test, deep_test = self.water_production(ars, nvs, df_list, orbit_whole, estimated_conditions)
            size_mid = math.pow(np.sqrt(WPR_max/max_test)**3/1.5**2, 1/3)*2
            print('size = ', size_mid, 'MAPE = ', score)
            size_list.append(size_mid)
        size_list = np.array(size_list)
        size = np.mean(size_list)
        return size

In [ ]:
tab_files1 = 'water_c_2002_y1_juels_hovorcem.tab'
tab_files2 = '2002_y1_juels_hovorcem.txt'

Size_cal = Size_inverse(tab_files1, tab_files2)
Size_cal.cal()

In [ ]:
# This code is designed for the inversion of the nucleus size of C/2002 Y1. 
# If applied to other comets, the input files must be modified, 
# and the activity peak position in the class should be adjusted according to the water curve peak. 
# Otherwise, the code will fail to converge.

In [ ]:
# Due to the numerous fitting parameters, the annealing algorithm in this code runs for 30 cycles, 
# and the average value is taken as the reference for the inverted size. 
# The number of cycles or the parameters within the annealing algorithm can be modified to achieve a more precise size inversion.
# However, considering the trade-off between precision and computational time, we recommend avoiding excessive iterations.